# Explore the PEP extraction results on the Guardian Data

## Setup and helper functions

In [1]:
import json

from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp

import matplotlib.pyplot as plt

from pepbench.io import load_challenge_results_from_folder
from pepbench.io import convert_hz_to_ms
from pepbench.datasets import GuardianDataset

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
root_path = Path("../../")

In [3]:
deploy_type = "local"

config_dict = json.load(root_path.joinpath("config.json").open(encoding="utf-8"))

empkins_base_path = Path(config_dict[deploy_type]["empkins_path"])
guardian_base_path = Path(config_dict[deploy_type]["guardian_path"])
print(guardian_base_path)

\Users\sebas\Development\ResearchInternship\Data\2024_08_PEP_Benchmarking\Guardian_Dataset


#### Specify whether to save the results or not

In [4]:
save_results = True

In [5]:
result_path = root_path.joinpath("results")
result_path

WindowsPath('../../results')

In [6]:
dataset_guardian = GuardianDataset(guardian_base_path, use_cache=True, only_labeled=True)
fs_guardian = dataset_guardian.sampling_rate_icg
print(f"Sampling rate ICG: {fs_guardian}")
dataset_guardian

Sampling rate ICG: 500


GuardianDataset [106 groups/rows]

       participant          phase
   0       GDN0005          Pause
   1       GDN0005       Valsalva
   2       GDN0005  HoldingBreath
   3       GDN0005         TiltUp
   4       GDN0005       TiltDown
   ..          ...            ...
   101     GDN0029         TiltUp
   102     GDN0029       TiltDown
   103     GDN0030       Valsalva
   104     GDN0030  HoldingBreath
   105     GDN0030       TiltDown
   
   [106 rows x 2 columns]

In [7]:
algo_levels = ["q_wave_algorithm", "b_point_algorithm", "outlier_correction_algorithm"]
algo_level_mapping = dict(zip(algo_levels, ["Q-Wave Algorithm", "B-Point Algorithm", "Outlier Correction"]))

In [8]:
results_guardian = load_challenge_results_from_folder(
    result_path.joinpath("guardian_dataset_reference_q_wave").resolve(), index_cols_per_sample=["participant", "phase"]
)

In [9]:
results_guardian_b_point = results_guardian.per_sample.droplevel("q_wave_algorithm")[[("heartbeat_id", "reference"), ("b_point_sample", "estimated"), ("b_point_sample", "reference")]]
results_guardian_b_point = results_guardian_b_point.reset_index()
results_guardian_b_point

b_point_algorithm outlier_correction_algorithm  \
                                                                       
0       arbol2017-isoelectric-crossings              forouzanfar2018   
1       arbol2017-isoelectric-crossings              forouzanfar2018   
2       arbol2017-isoelectric-crossings              forouzanfar2018   
3       arbol2017-isoelectric-crossings              forouzanfar2018   
4       arbol2017-isoelectric-crossings              forouzanfar2018   
...                                 ...                          ...   
198325                        stern1985                         none   
198326                        stern1985                         none   
198327                        stern1985                         none   
198328                        stern1985                         none   
198329                        stern1985                         none   

       participant     phase level_4 heartbeat_id b_point_sample            
                                        reference      estimated reference  
0          GDN0005     Pause       0            0          558.0       568  
1          GDN0005     Pause       1            1         1009.0      1025  
2          GDN0005     Pause       2            2         1463.0      1456  
3          GDN0005     Pause       3            3         1900.0      1888  
4          GDN0005     Pause       4            4         2320.0      2293  
...            ...       ...     ...          ...            ...       ...  
198325     GDN0030  TiltDown      51           51        27277.0     27278  
198326     GDN0030  TiltDown      52           52        27788.0     27815  
198327     GDN0030  TiltDown      53           53        28345.0     28345  
198328     GDN0030  TiltDown      54           54        28846.0     28867  
198329     GDN0030  TiltDown      55           55        29364.0     29389  

[198330 rows x 8 columns]

In [10]:
results_guardian_b_point.columns = results_guardian_b_point.columns.to_flat_index().str.join("")
results_guardian_b_point = results_guardian_b_point.drop(columns=["level_4"])
results_guardian_b_point

b_point_algorithm outlier_correction_algorithm  \
0       arbol2017-isoelectric-crossings              forouzanfar2018   
1       arbol2017-isoelectric-crossings              forouzanfar2018   
2       arbol2017-isoelectric-crossings              forouzanfar2018   
3       arbol2017-isoelectric-crossings              forouzanfar2018   
4       arbol2017-isoelectric-crossings              forouzanfar2018   
...                                 ...                          ...   
198325                        stern1985                         none   
198326                        stern1985                         none   
198327                        stern1985                         none   
198328                        stern1985                         none   
198329                        stern1985                         none   

       participant     phase  heartbeat_idreference  b_point_sampleestimated  \
0          GDN0005     Pause                      0                    558.0   
1          GDN0005     Pause                      1                   1009.0   
2          GDN0005     Pause                      2                   1463.0   
3          GDN0005     Pause                      3                   1900.0   
4          GDN0005     Pause                      4                   2320.0   
...            ...       ...                    ...                      ...   
198325     GDN0030  TiltDown                     51                  27277.0   
198326     GDN0030  TiltDown                     52                  27788.0   
198327     GDN0030  TiltDown                     53                  28345.0   
198328     GDN0030  TiltDown                     54                  28846.0   
198329     GDN0030  TiltDown                     55                  29364.0   

        b_point_samplereference  
0                           568  
1                          1025  
2                          1456  
3                          1888  
4                          2293  
...                         ...  
198325                    27278  
198326                    27815  
198327                    28345  
198328                    28867  
198329                    29389  

[198330 rows x 7 columns]

In [11]:
results_guardian_b_point["b_point_algorithm_combi"] = results_guardian_b_point["b_point_algorithm"] + "_" + results_guardian_b_point["outlier_correction_algorithm"]
results_guardian_b_point = results_guardian_b_point.drop(columns=["b_point_algorithm", "outlier_correction_algorithm"])
results_guardian_b_point.reindex(level=["participant", "phase"])

participant     phase  heartbeat_idreference  b_point_sampleestimated  \
0          GDN0005     Pause                      0                    558.0   
1          GDN0005     Pause                      1                   1009.0   
2          GDN0005     Pause                      2                   1463.0   
3          GDN0005     Pause                      3                   1900.0   
4          GDN0005     Pause                      4                   2320.0   
...            ...       ...                    ...                      ...   
198325     GDN0030  TiltDown                     51                  27277.0   
198326     GDN0030  TiltDown                     52                  27788.0   
198327     GDN0030  TiltDown                     53                  28345.0   
198328     GDN0030  TiltDown                     54                  28846.0   
198329     GDN0030  TiltDown                     55                  29364.0   

        b_point_samplereference  \
0                           568   
1                          1025   
2                          1456   
3                          1888   
4                          2293   
...                         ...   
198325                    27278   
198326                    27815   
198327                    28345   
198328                    28867   
198329                    29389   

                                b_point_algorithm_combi  
0       arbol2017-isoelectric-crossings_forouzanfar2018  
1       arbol2017-isoelectric-crossings_forouzanfar2018  
2       arbol2017-isoelectric-crossings_forouzanfar2018  
3       arbol2017-isoelectric-crossings_forouzanfar2018  
4       arbol2017-isoelectric-crossings_forouzanfar2018  
...                                                 ...  
198325                                   stern1985_none  
198326                                   stern1985_none  
198327                                   stern1985_none  
198328                                   stern1985_none  
198329                                   stern1985_none  

[198330 rows x 6 columns]

In [12]:
print("Possible B-Point algorithm combinations: ")
algo_list = results_guardian_b_point["b_point_algorithm_combi"].unique()
print(f"There are {len(algo_list)} possible algorithm combinations:")
algo_list

Possible B-Point algorithm combinations: 
There are 30 possible algorithm combinations:


array(['arbol2017-isoelectric-crossings_forouzanfar2018',
       'arbol2017-isoelectric-crossings_linear-interpolation',
       'arbol2017-isoelectric-crossings_none',
       'arbol2017-second-derivative_forouzanfar2018',
       'arbol2017-second-derivative_linear-interpolation',
       'arbol2017-second-derivative_none',
       'arbol2017-third-derivative_forouzanfar2018',
       'arbol2017-third-derivative_linear-interpolation',
       'arbol2017-third-derivative_none',
       'debski1993-second-derivative_forouzanfar2018',
       'debski1993-second-derivative_linear-interpolation',
       'debski1993-second-derivative_none', 'drost2022_forouzanfar2018',
       'drost2022_linear-interpolation', 'drost2022_none',
       'forounzafar2018_forouzanfar2018',
       'forounzafar2018_linear-interpolation', 'forounzafar2018_none',
       'lozano2007-linear-regression_forouzanfar2018',
       'lozano2007-linear-regression_linear-interpolation',
       'lozano2007-linear-regression_none',
    

In [13]:
print(f"Number of rows in the long dataframe: {results_guardian_b_point.shape[0]}")
print(f"Number of possible B-Point algorithm combinations: {len(algo_list)}")
print(f"Expected amount of rows in the wide datframe: {results_guardian_b_point.shape[0]/len(algo_list)}")

Number of rows in the long dataframe: 198330
Number of possible B-Point algorithm combinations: 30
Expected amount of rows in the wide datframe: 6611.0


### Convert the dataframe from the long to the wide format

In [14]:
# Pivot the DataFrame
df_guardian_pivot = results_guardian_b_point.pivot_table(
    index=['participant', 'phase', 'heartbeat_idreference', 'b_point_samplereference'],
    columns='b_point_algorithm_combi',
    values='b_point_sampleestimated'
).reset_index()

# Flatten the columns
df_guardian_pivot.columns.name = None
df_guardian_pivot.columns = [f'{col}' if isinstance(col, str) else f'{col[1]}' for col in df_guardian_pivot.columns]
df_guardian_pivot

participant          phase  heartbeat_idreference  \
0        GDN0005  HoldingBreath                      0   
1        GDN0005  HoldingBreath                      1   
2        GDN0005  HoldingBreath                      3   
3        GDN0005  HoldingBreath                      4   
4        GDN0005  HoldingBreath                      5   
...          ...            ...                    ...   
6239     GDN0030       Valsalva                     51   
6240     GDN0030       Valsalva                     52   
6241     GDN0030       Valsalva                     53   
6242     GDN0030       Valsalva                     54   
6243     GDN0030       Valsalva                     55   

      b_point_samplereference  \
0                         540   
1                         973   
2                        1741   
3                        2121   
4                        2499   
...                       ...   
6239                    27597   
6240                    28086   
6241                    28581   
6242                    29061   
6243                    29594   

      arbol2017-isoelectric-crossings_forouzanfar2018  \
0                                               565.0   
1                                               981.0   
2                                              1744.0   
3                                              2123.0   
4                                              2505.0   
...                                               ...   
6239                                          27581.0   
6240                                          28073.0   
6241                                          28591.0   
6242                                          29074.0   
6243                                          29591.0   

      arbol2017-isoelectric-crossings_linear-interpolation  \
0                                                 565.0      
1                                                 982.0      
2                                                1744.0      
3                                                2123.0      
4                                                2505.0      
...                                                 ...      
6239                                            27581.0      
6240                                            28073.0      
6241                                            28591.0      
6242                                            29074.0      
6243                                            29591.0      

      arbol2017-isoelectric-crossings_none  \
0                                    565.0   
1                                    941.0   
2                                   1744.0   
3                                   2123.0   
4                                   2505.0   
...                                    ...   
6239                               27581.0   
6240                               28073.0   
6241                               28591.0   
6242                               29074.0   
6243                               29591.0   

      arbol2017-second-derivative_forouzanfar2018  \
0                                           545.0   
1                                           946.0   
2                                          1701.0   
3                                          2098.0   
4                                          2462.0   
...                                           ...   
6239                                      27554.0   
6240                                      28043.0   
6241                                      28557.0   
6242                                      29045.0   
6243                                      29584.0   

      arbol2017-second-derivative_linear-interpolation  \
0                                                545.0   
1                                                946.0   
2                                               1701.0   
3                                               2098.0   
4

In [15]:
print(f"Max heartbeat id long dataframe: {max(results_guardian_b_point["heartbeat_idreference"])}")
print(f"Max heartbeat id wide dataframe: {max(df_guardian_pivot["heartbeat_idreference"])}")

Max heartbeat id long dataframe: 97
Max heartbeat id wide dataframe: 97


### Save the pivot dataframe

In [16]:
if save_results:
    df_guardian_pivot.to_csv(result_path.joinpath("pivot_dataframe_b_point/guardian_pivot_dataframe_b_point_all_algos.csv"))

### Convert samples to ms

In [17]:
df_guardian_pivot_ms = df_guardian_pivot.copy()
exclude_cols = ["participant", "condition", "phase", "heartbeat_idreference"]
df_guardian_pivot_ms.loc[:, ~df_guardian_pivot_ms.columns.isin(exclude_cols)] = df_guardian_pivot_ms.loc[:, ~df_guardian_pivot_ms.columns.isin(exclude_cols)].apply(lambda x: x * convert_hz_to_ms(fs_guardian))
df_guardian_pivot_ms                                                                                  

C:\Users\sebas\AppData\Local\Temp\ipykernel_11288\2339280947.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_guardian_pivot_ms.loc[:, ~df_guardian_pivot_ms.columns.isin(exclude_cols)] = df_guardian_pivot_ms.loc[:, ~df_guardian_pivot_ms.columns.isin(exclude_cols)].apply(lambda x: x * convert_hz_to_ms(fs_guardian))


participant          phase  heartbeat_idreference  \
0        GDN0005  HoldingBreath                      0   
1        GDN0005  HoldingBreath                      1   
2        GDN0005  HoldingBreath                      3   
3        GDN0005  HoldingBreath                      4   
4        GDN0005  HoldingBreath                      5   
...          ...            ...                    ...   
6239     GDN0030       Valsalva                     51   
6240     GDN0030       Valsalva                     52   
6241     GDN0030       Valsalva                     53   
6242     GDN0030       Valsalva                     54   
6243     GDN0030       Valsalva                     55   

      b_point_samplereference  \
0                      1080.0   
1                      1946.0   
2                      3482.0   
3                      4242.0   
4                      4998.0   
...                       ...   
6239                  55194.0   
6240                  56172.0   
6241                  57162.0   
6242                  58122.0   
6243                  59188.0   

      arbol2017-isoelectric-crossings_forouzanfar2018  \
0                                              1130.0   
1                                              1962.0   
2                                              3488.0   
3                                              4246.0   
4                                              5010.0   
...                                               ...   
6239                                          55162.0   
6240                                          56146.0   
6241                                          57182.0   
6242                                          58148.0   
6243                                          59182.0   

      arbol2017-isoelectric-crossings_linear-interpolation  \
0                                                1130.0      
1                                                1964.0      
2                                                3488.0      
3                                                4246.0      
4                                                5010.0      
...                                                 ...      
6239                                            55162.0      
6240                                            56146.0      
6241                                            57182.0      
6242                                            58148.0      
6243                                            59182.0      

      arbol2017-isoelectric-crossings_none  \
0                                   1130.0   
1                                   1882.0   
2                                   3488.0   
3                                   4246.0   
4                                   5010.0   
...                                    ...   
6239                               55162.0   
6240                               56146.0   
6241                               57182.0   
6242                               58148.0   
6243                               59182.0   

      arbol2017-second-derivative_forouzanfar2018  \
0                                          1090.0   
1                                          1892.0   
2                                          3402.0   
3                                          4196.0   
4                                          4924.0   
...                                           ...   
6239                                      55108.0   
6240                                      56086.0   
6241                                      57114.0   
6242                                      58090.0   
6243                                      59168.0   

      arbol2017-second-derivative_linear-interpolation  \
0                                               1090.0   
1                                               1892.0   
2                                               3402.0   
3                                               4196.0   
4

### Save the converted dataframe

In [18]:
if save_results:
    df_guardian_pivot_ms.to_csv(result_path.joinpath("pivot_dataframe_b_point/guardian_pivot_dataframe_b_point_all_algos_ms.csv"))